<a href="https://colab.research.google.com/github/hewenpang/finetune_path/blob/main/bert_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import  AutoTokenizer,AutoModelForSequenceClassification
!pip install datasets
from datasets import  load_dataset
import pandas as pd
from torch.utils.data import  DataLoader,Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
class My_dataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv('./ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]['review'],self.data.iloc[index]['label']

    def __len__(self):
        return len(self.data)

In [3]:
dataset = My_dataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [4]:
from torch.utils.data import random_split

train_data,test_data = random_split(dataset,lengths=[0.8,0.2])

print(train_data)

In [11]:
model  = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-uncased',num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

def collate_fn(batch):
    texts,labels = [],[]
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts,padding='max_length',max_length=128,truncation=True,return_tensors='pt')
    inputs['labels'] = torch.tensor(labels)
    return inputs

train_loader = DataLoader(train_data,batch_size=32,shuffle=True,collate_fn=collate_fn)
test_loader = DataLoader(test_data,batch_size=64,shuffle=True,collate_fn=collate_fn)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model  = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-uncased',num_labels=2)

In [6]:
next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101,  100,  100,  ..., 1743, 1989,  102],
        [ 101,  100,  100,  ...,    0,    0,    0],
        [ 101, 1802,  100,  ...,    0,    0,    0],
        ...,
        [ 101, 1744,  100,  ...,    0,    0,    0],
        [ 101,  100,  100,  ...,    0,    0,    0],
        [ 101,  100,  100,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 1])}

In [7]:
from torch.optim import Adam

optimizer = Adam(model.parameters(),lr=2e-5)


In [18]:
!pip install evaluate
import  evaluate
metric = evaluate.combine(['accuracy','f1'])
def eval():
    model.eval()
    with torch.inference_mode():
        for batch in test_loader:
            if torch.cuda.is_available():
              batch = {k:v.cuda() for k,v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits,dim=-1)
            metric.add_batch(pred,batch['labels'])
    model.train()
    return metric.compute()


In [12]:
# 直接用与训练模型进行推理 并且evaluate
if torch.cuda.is_available():
  model = model.cuda()
for batch in test_loader:
    if torch.cuda.is_available():
        batch = {k:v.cuda() for k,v in batch.items()}
    output = model(**batch)
    pred = torch.argmax(output.logits,dim=-1)
    metric.add_batch(pred,batch['labels'])
print(metric.compute())

{'accuracy': 0.6535737282678686, 'f1': 0.7756463719766472}


In [21]:
def get_model_params(model):
    return {name: param.data.clone() for name, param in model.named_parameters()}

def compute_param_diff(params_before, params_after):
    diff = {}
    for name in params_before:
        diff[name] = torch.abs(params_after[name] - params_before[name]).mean().item()
    return diff

def train(epoch=3,log_steps=100):
    global_step = 0
    params_before = get_model_params(model)
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
              batch = {k:v.cuda() for k,v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_steps == 0:
                print(f"ep:{ep},global_step:{global_step},loss:{output.loss.item():.4f}")
            global_step+=1
        acc = eval()
        print(f"ep:{ep},acc:{acc}")
    params_after = get_model_params(model)
    param_diff = compute_param_diff(params_before, params_after)
    print("Parameter changes after training:")
    for name, diff in param_diff.items():
        print(f"{name}: {diff:.6f}")
train()

ep:0,global_step:0,loss:0.6387
ep:0,global_step:100,loss:0.6841
ep:0,acc:{'accuracy': 0.6535737282678686, 'f1': 0.7756463719766472}
ep:1,global_step:200,loss:0.6992
ep:1,global_step:300,loss:0.6762
ep:1,acc:{'accuracy': 0.6535737282678686, 'f1': 0.7756463719766472}
ep:2,global_step:400,loss:0.6611
ep:2,global_step:500,loss:0.6556
ep:2,acc:{'accuracy': 0.6535737282678686, 'f1': 0.7756463719766472}
Parameter changes after training:
bert.embeddings.word_embeddings.weight: 0.000000
bert.embeddings.position_embeddings.weight: 0.000000
bert.embeddings.token_type_embeddings.weight: 0.000000
bert.embeddings.LayerNorm.weight: 0.000000
bert.embeddings.LayerNorm.bias: 0.000000
bert.encoder.layer.0.attention.self.query.weight: 0.000000
bert.encoder.layer.0.attention.self.query.bias: 0.000000
bert.encoder.layer.0.attention.self.key.weight: 0.000000
bert.encoder.layer.0.attention.self.key.bias: 0.000000
bert.encoder.layer.0.attention.self.value.weight: 0.000000
bert.encoder.layer.0.attention.self.va